<a href="https://colab.research.google.com/github/Amplil/pytorch/blob/master/7_2_torchtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.2 torchtextでのDataset、DataLoaderの実装方法

- 本ファイルでは、torchtextを使用してDatasetおよびDataLoaderを実装する方法を解説します。


※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.2 学習目標

1.	torchtextを用いてDatasetおよびDataLoaderの実装ができる

# 事前準備

- 書籍の指示に従い、本章で使用するデータを用意します

- torchtextをインストールします

- pip install torchtext



# 1 . 前処理と単語分割の関数を実装


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")

In [13]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced/7_nlp_sentiment_transformer


In [8]:
!pip install --target "/content/drive/MyDrive/Colab Notebooks/packages" torchtext

     |████████████████████████████████| 7.1MB 5.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 735.5MB 25kB/s 
     |████████████████████████████████| 15.3MB 262kB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 153kB 55.0MB/s 
     |████████████████████████████████| 61kB 6.9MB/s 
     |████████████████████████████████| 184kB 64.0MB/s 
     |████████████████████████████████| 153kB 55.1MB/s 
ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.20.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [3]:
# 単語分割にはJanomeを使用
from janome.tokenizer import Tokenizer

j_t = Tokenizer()


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


In [4]:
# 前処理として正規化をする関数を定義
import re


def preprocessing_text(text):
    # 半角・全角の統一
    # 今回は無視

    # 英語の小文字化
    # 今回はここでは無視
    # output = output.lower()

    # 改行、半角スペース、全角スペースを削除
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 数字文字の一律「0」化
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字

    # 記号と数字の除去
    # 今回は無視。半角記号,数字,英字
    # 今回は無視。全角記号

    # 特定文字を正規表現で置換する
    # 今回は無視

    return text


In [5]:
# 前処理とJanomeの単語分割を合わせた関数を定義する


def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 前処理の正規化
    ret = tokenizer_janome(text)  # Janomeの単語分割

    return ret


# 動作確認
text = "昨日は とても暑く、気温が36度もあった。"
print(tokenizer_with_preprocessing(text))


['昨日', 'は', 'とても', '暑く', '、', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


# 2. 文章データの読み込み

In [8]:
import torchtext

# tsvやcsvデータを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# 文章とラベルの両方に用意します

max_length = 25
TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリー（単語集：後で解説）に追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を先頭に用意するかどうか
# fix_length：全部の文章を指定した長さと同じになるように、paddingします


In [15]:
# data.TabularDataset 詳細
# https://torchtext.readthedocs.io/en/latest/examples.html?highlight=data.TabularDataset.splits

# フォルダ「data」から各tsvファイルを読み込み、Datasetにします
# 1行がTEXTとLABELで区切られていることをfieldsで指示します
train_ds, val_ds, test_ds = torchtext.legacy.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])


# 動作確認
print('訓練データの数', len(train_ds))
print('1つ目の訓練データ', vars(train_ds[0]))
print('2つ目の訓練データ', vars(train_ds[1]))


訓練データの数 4
1つ目の訓練データ {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
2つ目の訓練データ {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


# 単語の数値化

In [16]:
# ボキャブラリーを作成します
# 訓練データtrainの単語からmin_freq以上の頻度の単語を使用してボキャブラリー（単語集）を構築
TEXT.build_vocab(train_ds, min_freq=1)

# 訓練データ内の単語と頻度を出力(頻度min_freqより大きいものが出力されます)
TEXT.vocab.freqs  # 出力させる


Counter({'0': 1,
         '、': 3,
         '。': 4,
         'い': 1,
         'いる': 2,
         'か': 2,
         'から': 1,
         'が': 3,
         'し': 3,
         'する': 1,
         'その': 1,
         'た': 1,
         'て': 2,
         'で': 1,
         'です': 1,
         'と': 5,
         'な': 4,
         'に': 1,
         'に対して': 1,
         'の': 4,
         'は': 1,
         'まし': 1,
         'ます': 2,
         'を': 3,
         'クラス': 1,
         'ネガティブ': 1,
         'ポジティブ': 1,
         'モデル': 1,
         'レビュー': 1,
         '値': 1,
         '処理': 1,
         '分類': 2,
         '取り組み': 1,
         '商品': 1,
         '女性': 1,
         '女王': 1,
         '好き': 1,
         '姫': 1,
         '学習': 1,
         '文章': 4,
         '本章': 2,
         '構築': 1,
         '機械': 1,
         '王': 1,
         '王子': 1,
         '男性': 1,
         '短い': 1,
         '自然': 1,
         '言語': 1,
         '評価': 2})

In [17]:
# ボキャブラリーの単語をidに変換した結果を出力。
# 頻度がmin_freqより小さい場合は未知語<unk>になる

TEXT.vocab.stoi  # 出力。string to identifiers 文字列をidへ


defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fcdb7c94790>>,
            {'0': 18,
             '<pad>': 1,
             '<unk>': 0,
             '、': 7,
             '。': 3,
             'い': 19,
             'いる': 11,
             'か': 12,
             'から': 20,
             'が': 8,
             'し': 9,
             'する': 21,
             'その': 22,
             'た': 23,
             'て': 13,
             'で': 24,
             'です': 25,
             'と': 2,
             'な': 4,
             'に': 26,
             'に対して': 27,
             'の': 5,
             'は': 28,
             'まし': 29,
             'ます': 14,
             'を': 10,
             'クラス': 30,
             'ネガティブ': 31,
             'ポジティブ': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '分類': 15,
             '取り組み': 37,
             '商品': 38,
             '女性': 39,
             '女王': 40,
             '好き': 41,
    

# DataLoaderの作成

In [19]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.legacy.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.legacy.data.Iterator(
    val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.legacy.data.Iterator(
    test_ds, batch_size=2, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])


以上